In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

In [2]:
# make sure pandas is version 1.0 or higher
# make sure networkx is verion 2.4 or higher
print(pd.__version__)
print(nx.__version__)

1.5.3
2.8.4


In [3]:
from ema_workbench import (
    Model,
    Policy,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator,
)
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time



In [4]:
ema_logging.log_to_stderr(ema_logging.INFO)

# choose problem formulation number, between 0-5
# each problem formulation has its own list of outcomes
dike_model, planning_steps = get_model_for_problem_formulation(6)

In [5]:
# enlisting uncertainties, their types (RealParameter/IntegerParameter/CategoricalParameter), lower boundary, and upper boundary
import copy

for unc in dike_model.uncertainties:
    print(repr(unc))

uncertainties = copy.deepcopy(dike_model.uncertainties)

CategoricalParameter('discount rate 0', [0, 1, 2, 3])
CategoricalParameter('discount rate 1', [0, 1, 2, 3])
CategoricalParameter('discount rate 2', [0, 1, 2, 3])
IntegerParameter('A.0_ID flood wave shape', 0, 132, resolution=None, default=None, variable_name=['A.0_ID flood wave shape'], pff=False)
RealParameter('A.1_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.1_Bmax'], pff=False)
RealParameter('A.1_pfail', 0, 1, resolution=None, default=None, variable_name=['A.1_pfail'], pff=False)
CategoricalParameter('A.1_Brate', [0, 1, 2])
RealParameter('A.2_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.2_Bmax'], pff=False)
RealParameter('A.2_pfail', 0, 1, resolution=None, default=None, variable_name=['A.2_pfail'], pff=False)
CategoricalParameter('A.2_Brate', [0, 1, 2])
RealParameter('A.3_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.3_Bmax'], pff=False)
RealParameter('A.3_pfail', 0, 1, resolution=None, default=None, variable_name=['A.3_pfai

In [6]:
# enlisting policy levers, their types (RealParameter/IntegerParameter), lower boundary, and upper boundary
for policy in dike_model.levers:
    print(repr(policy))

levers = copy.deepcopy(dike_model.levers)

IntegerParameter('0_RfR 0', 0, 1, resolution=None, default=None, variable_name=['0_RfR 0'], pff=False)
IntegerParameter('0_RfR 1', 0, 1, resolution=None, default=None, variable_name=['0_RfR 1'], pff=False)
IntegerParameter('0_RfR 2', 0, 1, resolution=None, default=None, variable_name=['0_RfR 2'], pff=False)
IntegerParameter('1_RfR 0', 0, 1, resolution=None, default=None, variable_name=['1_RfR 0'], pff=False)
IntegerParameter('1_RfR 1', 0, 1, resolution=None, default=None, variable_name=['1_RfR 1'], pff=False)
IntegerParameter('1_RfR 2', 0, 1, resolution=None, default=None, variable_name=['1_RfR 2'], pff=False)
IntegerParameter('2_RfR 0', 0, 1, resolution=None, default=None, variable_name=['2_RfR 0'], pff=False)
IntegerParameter('2_RfR 1', 0, 1, resolution=None, default=None, variable_name=['2_RfR 1'], pff=False)
IntegerParameter('2_RfR 2', 0, 1, resolution=None, default=None, variable_name=['2_RfR 2'], pff=False)
IntegerParameter('3_RfR 0', 0, 1, resolution=None, default=None, variable

In [7]:
# enlisting outcomes
for outcome in dike_model.outcomes:
    print(repr(outcome))

ScalarOutcome('A.1_Expected Annual Damage', variable_name=('A.1_Expected Annual Damage',), function=<function sum_over at 0x0000024D7B2DE9D0>)
ScalarOutcome('A.1_Dike Investment Costs', variable_name=('A.1_Dike Investment Costs',), function=<function sum_over at 0x0000024D7B2DE9D0>)
ScalarOutcome('A.1_Expected Number of Deaths', variable_name=('A.1_Expected Number of Deaths',), function=<function sum_over at 0x0000024D7B2DE9D0>)
ScalarOutcome('A.2_Expected Annual Damage', variable_name=('A.2_Expected Annual Damage',), function=<function sum_over at 0x0000024D7B2DE9D0>)
ScalarOutcome('A.2_Dike Investment Costs', variable_name=('A.2_Dike Investment Costs',), function=<function sum_over at 0x0000024D7B2DE9D0>)
ScalarOutcome('A.2_Expected Number of Deaths', variable_name=('A.2_Expected Number of Deaths',), function=<function sum_over at 0x0000024D7B2DE9D0>)
ScalarOutcome('A.3_Expected Annual Damage', variable_name=('A.3_Expected Annual Damage',), function=<function sum_over at 0x0000024D7B

In [8]:
# running the model through EMA workbench
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(scenarios=50, policies=4)

[MainProcess/INFO] pool started with 8 workers
[MainProcess/INFO] performing 50 scenarios * 4 policies * 1 model(s) = 200 experiments
100%|████████████████████████████████████████| 200/200 [01:18<00:00,  2.55it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [9]:
# observing the simulation runs
experiments, outcomes = results
print(outcomes.keys())
experiments

dict_keys(['A.1_Expected Annual Damage', 'A.1_Dike Investment Costs', 'A.1_Expected Number of Deaths', 'A.2_Expected Annual Damage', 'A.2_Dike Investment Costs', 'A.2_Expected Number of Deaths', 'A.3_Expected Annual Damage', 'A.3_Dike Investment Costs', 'A.3_Expected Number of Deaths', 'A.4_Expected Annual Damage', 'A.4_Dike Investment Costs', 'A.4_Expected Number of Deaths', 'A.5_Expected Annual Damage', 'A.5_Dike Investment Costs', 'A.5_Expected Number of Deaths', 'RfR Total Costs', 'Expected Evacuation Costs'])


,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,A.3_pfail,...,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,EWS_DaysToThreat,scenario,policy,model
0,22,144.182313,10.0,0.555674,183.713884,10.0,0.038645,164.311828,1.5,0.333012,...,6,0,4,3,9,10,2,4,0,dikesnet
1,16,126.317796,1.0,0.351584,214.684088,1.0,0.109075,272.076745,10.0,0.820712,...,6,0,4,3,9,10,2,5,0,dikesnet
2,44,157.506901,10.0,0.431080,229.503152,10.0,0.420870,313.734331,1.0,0.433078,...,6,0,4,3,9,10,2,6,0,dikesnet
3,2,135.809522,10.0,0.598117,123.522873,1.5,0.552705,184.593358,1.0,0.545527,...,6,0,4,3,9,10,2,7,0,dikesnet
4,37,74.649290,10.0,0.206949,306.688914,10.0,0.231164,277.297786,1.0,0.197778,...,6,0,4,3,9,10,2,8,0,dikesnet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,118,291.125479,1.5,0.752910,295.292451,1.0,0.194411,81.796321,1.5,0.235365,...,9,4,7,7,3,5,4,49,3,dikesnet
196,129,255.357768,1.0,0.871191,51.758704,1.5,0.750996,118.638922,1.5,0.658603,...,9,4,7,7,3,5,4,50,3,dikesnet
197,62,52.984769,1.5,0.685489,304.480542,10.0,0.268563,66.514630,1.5,0.885379,...,9,4,7,7,3,5,4,51,3,dikesnet
198,127,88.591216,1.5,0.364936,254.673395,1.5,0.597781,47.852225,1.0,0.968764,...,9,4,7,7,3,5,4,52,3,dikesnet


In [10]:
# only works because we have scalar outcomes
pd.DataFrame(outcomes)

,A.1_Expected Annual Damage,A.1_Dike Investment Costs,A.1_Expected Number of Deaths,A.2_Expected Annual Damage,A.2_Dike Investment Costs,A.2_Expected Number of Deaths,A.3_Expected Annual Damage,A.3_Dike Investment Costs,A.3_Expected Number of Deaths,A.4_Expected Annual Damage,A.4_Dike Investment Costs,A.4_Expected Number of Deaths,A.5_Expected Annual Damage,A.5_Dike Investment Costs,A.5_Expected Number of Deaths,RfR Total Costs,Expected Evacuation Costs
0,0.0,1.489975e+08,0.0,0.000000e+00,1.438745e+08,0.000000,4.498158e+06,7.622963e+07,0.001462,0.0,2.444716e+07,0.0,0.000000e+00,1.761661e+08,0.000000,1.761500e+09,198.579748
1,0.0,1.489975e+08,0.0,0.000000e+00,1.438745e+08,0.000000,0.000000e+00,7.622963e+07,0.000000,0.0,2.444716e+07,0.0,2.227647e+06,1.761661e+08,0.000233,1.761500e+09,83.522415
2,0.0,1.489975e+08,0.0,0.000000e+00,1.438745e+08,0.000000,1.561096e+06,7.622963e+07,0.000315,0.0,2.444716e+07,0.0,0.000000e+00,1.761661e+08,0.000000,1.761500e+09,41.929005
3,0.0,1.489975e+08,0.0,0.000000e+00,1.438745e+08,0.000000,1.207968e+06,7.622963e+07,0.000317,0.0,2.444716e+07,0.0,0.000000e+00,1.761661e+08,0.000000,1.761500e+09,41.953656
4,0.0,1.489975e+08,0.0,0.000000e+00,1.438745e+08,0.000000,1.196617e+07,7.622963e+07,0.003870,0.0,2.444716e+07,0.0,0.000000e+00,1.761661e+08,0.000000,1.761500e+09,531.296830
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0.0,3.428850e+08,0.0,2.125245e+07,1.068602e+08,0.001857,0.000000e+00,9.041765e+07,0.000000,0.0,5.333609e+07,0.0,0.000000e+00,1.339162e+08,0.000000,4.080000e+08,1003.147804
196,0.0,3.428850e+08,0.0,0.000000e+00,1.068602e+08,0.000000,0.000000e+00,9.041765e+07,0.000000,0.0,5.333609e+07,0.0,0.000000e+00,1.339162e+08,0.000000,4.080000e+08,0.000000
197,0.0,3.428850e+08,0.0,1.883104e+07,1.068602e+08,0.001859,0.000000e+00,9.041765e+07,0.000000,0.0,5.333609e+07,0.0,0.000000e+00,1.339162e+08,0.000000,4.080000e+08,1003.597126
198,0.0,3.428850e+08,0.0,0.000000e+00,1.068602e+08,0.000000,0.000000e+00,9.041765e+07,0.000000,0.0,5.333609e+07,0.0,0.000000e+00,1.339162e+08,0.000000,4.080000e+08,0.000000


In [11]:
# defining specific policies
# for example, policy 1 is about extra protection in upper boundary
# policy 2 is about extra protection in lower boundary
# policy 3 is extra protection in random locations


def get_do_nothing_dict():
    return {l.name: 0 for l in dike_model.levers}


policies = [
    Policy(
        "Policy 1",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 1, "0_RfR 1": 1, "0_RfR 2": 1, "A.1_DikeIncrease 0": 5}
        )
    ),
    Policy(
        "Policy 2",
        **dict(
            get_do_nothing_dict(),
            **{"4_RfR 0": 1, "4_RfR 1": 1, "4_RfR 2": 1, "A.5_DikeIncrease 0": 5}
        )
    ),
    Policy(
        "Policy 3",
        **dict(
            get_do_nothing_dict(),
            **{"1_RfR 0": 1, "2_RfR 1": 1, "3_RfR 2": 1, "A.3_DikeIncrease 0": 5}
        )
    ),
    Policy(
        "Baseline",
        **dict(
            get_do_nothing_dict(),
            **{"1_RfR 0": 0, "2_RfR 1": 0, "3_RfR 2": 0, "A.3_DikeIncrease 0": 0}
        )
    )
    ,
    Policy(
        "Policy DImax",
        **dict(
            get_do_nothing_dict(),
            **{
               "A.1_DikeIncrease 0": 5,"A.2_DikeIncrease 0": 5,"A.3_DikeIncrease 0": 5,"A.4_DikeIncrease 0": 5, "A.5_DikeIncrease 0": 5}
        )
    )
    ,
        Policy(
        "Policy RfRmax",
        **dict(
            get_do_nothing_dict(),
            **{
               "0_RfR 0": 1, "1_RfR 0": 1, "2_RfR 0": 1, "3_RfR 0": 1, "4_RfR 0": 1, "5_RfR 0": 1}
        )
    )
]

In [12]:
# pass the policies list to EMA workbench experiment runs
n_scenarios = 100
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(n_scenarios, policies)

[MainProcess/INFO] pool started with 8 workers
[MainProcess/INFO] performing 100 scenarios * 6 policies * 1 model(s) = 600 experiments
100%|████████████████████████████████████████| 600/600 [03:16<00:00,  3.05it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [13]:
experiments, outcomes = results
results.head()

In [15]:
from funs_viz import tidy_results, merge_results


results_df = tidy_results(results)
results_df.to_csv("results/test_1.csv")

print(results_df.columns)
results_df.head()

Index(['A.0_ID flood wave shape', 'discount rate 0', 'discount rate 1',
       'discount rate 2', 'EWS_DaysToThreat', 'scenario', 'policy', 'model',
       'RfR Total Costs', 'Expected Evacuation Costs', 'Dike ring', 'Bmax',
       'Brate', 'pfail', 'RfR 0', 'RfR 1', 'RfR 2', 'DikeIncrease 0',
       'DikeIncrease 1', 'DikeIncrease 2', 'Expected Annual Damage',
       'Dike Investment Costs', 'Expected Number of Deaths'],
      dtype='object')
